In [1]:
import datasets
from datasets import load_dataset, inspect_dataset, load_dataset_builder, VerificationMode

/home/onomaai/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
builder = load_dataset_builder(
    './wmt14/wmt_utils.py',
    language_pair = ('fr', 'en'),
    subsets = {
        datasets.Split.TRAIN: ["europarl_v7", "newscommentary_v10"],
        datasets.Split.VALIDATION: ['newstest2013'],
        datasets.Split.TEST: ['newstest2014']
    },
    cache_dir = '/data2/brian/.cache/dataset'
)
# datasets.DatasetBuilder class

/home/onomaai/anaconda3/envs/brian/lib/python3.10/site-packages/datasets/load.py:926: FutureWarning: The repository for wmt_utils contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at ./wmt14/wmt_utils.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
builder.download_and_prepare(verification_mode=VerificationMode.NO_CHECKS)

In [4]:
dataset = builder.as_dataset()

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 2201939
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3003
    })
})

In [8]:
dataset['validation']['translation']

[{'en': 'A Republican strategy to counter the re-election of Obama',
  'fr': "Une stratégie républicaine pour contrer la réélection d'Obama"},
 {'en': 'Republican leaders justified their policy by the need to combat electoral fraud.',
  'fr': 'Les dirigeants républicains justifièrent leur politique par la nécessité de lutter contre la fraude électorale.'},
 {'en': 'However, the Brennan Centre considers this a myth, stating that electoral fraud is rarer in the United States than the number of people killed by lightning.',
  'fr': 'Or, le Centre Brennan considère cette dernière comme un mythe, affirmant que la fraude électorale est plus rare aux États-Unis que le nombre de personnes tuées par la foudre.'},
 {'en': 'Indeed, Republican lawyers identified only 300 cases of electoral fraud in the United States in a decade.',
  'fr': "D'ailleurs, les avocats républicains n'ont recensé que 300 cas de fraude électorale aux États-Unis en dix ans."},
 {'en': 'One thing is certain: these new provi

In [40]:
with open('fr-en/europarl-v7.fr-en.en', 'r') as f:
    en_text = f.readlines()
with open('fr-en/europarl-v7.fr-en.fr', 'r') as f:
    fr_text = f.readlines()

In [42]:
sent_data = Dataset.from_dict({'en': en_text, 'fr': fr_text})

In [45]:
sent_data.save_to_disk('fr-en/hf')

Saving the dataset (2/2 shards): 100%|██████████| 2007723/2007723 [00:01<00:00, 1632848.03 examples/s]


In [37]:
from datasets import Dataset
sent_data = Dataset([{'en': en.strip(), 'fr': fr.strip()} for en, fr in zip(en_text, fr_text)])
sent_data.save_to_disk('fr-en/hf')

In [28]:
import os
file_names = list(map(os.path.basename, os.listdir('French-English/English')))

In [29]:
en_path = lambda x: f"French-English/English/{x}"
fr_path = lambda x: f"French-English/French/{x}"

In [30]:
with open(en_path(file_names[2])) as f:
    text = f.read()

In [24]:
from bs4 import BeautifulSoup
import unicodedata
cleantext = BeautifulSoup(text, "lxml").text
cleantext = unicodedata.normalize('NFKC', cleantext)
cleantext = cleantext.replace(u'\\u0027', "'")
cleantext

'Look East To Save Europe\'s Social Market\n\n\nAs expansion of the EU approaches, many Europeans see in it only things to be feared: masses of economic migrants, and poor countries demanding subsidies.\nBut Europe\'s new eastern members can also act as a beacon for the Union, as Jacques Rupnik suggests.\n\nIt is often argued that Continental Europe\'s social and economic model, which seeks to combine competitiveness with solidarity, is the glue that binds the European Union together, as well as distinguishing Europe from the American (or Anglo-Saxon) free-market model.\nClearly, Europe\'s answer to globalization is that certain spheres of social life-say, healthcare, education, the environment, or culture-cannot be left to the whip of the market.\n\nOn the surface it seems that Europe\'s steady integration proceeded in parallel with the development of the welfare state.\nBut this is misleading: the European social model is, in fact, part and parcel of the identity of the EU member sta

In [25]:
print(cleantext)

Look East To Save Europe's Social Market


As expansion of the EU approaches, many Europeans see in it only things to be feared: masses of economic migrants, and poor countries demanding subsidies.
But Europe's new eastern members can also act as a beacon for the Union, as Jacques Rupnik suggests.

It is often argued that Continental Europe's social and economic model, which seeks to combine competitiveness with solidarity, is the glue that binds the European Union together, as well as distinguishing Europe from the American (or Anglo-Saxon) free-market model.
Clearly, Europe's answer to globalization is that certain spheres of social life-say, healthcare, education, the environment, or culture-cannot be left to the whip of the market.

On the surface it seems that Europe's steady integration proceeded in parallel with the development of the welfare state.
But this is misleading: the European social model is, in fact, part and parcel of the identity of the EU member states more than of

In [33]:
import re
text_preprocessed = re.sub('\n+', ' ', re.sub('\<.*\>', '', text))
text_preprocessed

'Look East To Save Europe\\u0027s Social Market As expansion of the EU approaches, many Europeans see in it only things to be feared: masses of economic migrants, and poor countries demanding subsidies. But Europe\'s new eastern members can also act as a beacon for the Union, as Jacques Rupnik suggests. It is often argued that Continental Europe\'s social and economic model, which seeks to combine competitiveness with solidarity, is the glue that binds the European Union together, as well as distinguishing Europe from the American (or Anglo-Saxon) free-market model. Clearly, Europe\'s answer to globalization is that certain spheres of social life-say, healthcare, education, the environment, or culture-cannot be left to the whip of the market. On the surface it seems that Europe\'s steady integration proceeded in parallel with the development of the welfare state. But this is misleading: the European social model is, in fact, part and parcel of the identity of the EU member states more 

In [34]:
def read_and_clean_doc(path):
    with open(path, 'r') as f:
        text = f.read()
    text = BeautifulSoup(text, "lxml").text
    text = unicodedata.normalize('NFKC', text)
    text = text.replace(u'\\u0027', "'")
    text = re.sub('\n+', ' ', re.sub('\<.*\>', '', text))
    return text

In [8]:
read_and_clean_doc(fr_path(file_names[0]))

NameError: name 'unicodedata' is not defined

In [14]:
en_path(file_names[0])

'French-English/English/bush-inherits-the-wind.txt'

In [32]:
read_and_clean_doc(fr_path(file_names[1]))

NameError: name 're' is not defined

In [48]:
en_doc = [read_and_clean_doc(en_path(path)) for path in file_names]
fr_doc = [read_and_clean_doc(fr_path(path)) for path in file_names]

In [51]:
doc_data = Dataset.from_dict({'en': en_doc, 'fr': fr_doc})

In [53]:
doc_data.save_to_disk('French-English/hf')

Saving the dataset (1/1 shards): 100%|██████████| 4794/4794 [00:00<00:00, 92718.51 examples/s]
